In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = './data/'

In [3]:
currency_pair = 'Eur_Usd'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [5]:
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis=1, join='inner').max(axis=1)
    atr = tr.rolling(lookback).mean()

    plus_di = 100 * (plus_dm.ewm(alpha=1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha=1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha=1/lookback).mean()

    return adx_smooth


df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
df['ema25'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema200,ema100,ema50,ema25,adx
0,2021-04-01 07:10:00,1.17239,1.17334,1.17239,1.17316,1.17252,1.17348,1.17252,1.17330,1.17246,1.17341,1.17246,1.17323,662,1.172286,1.172304,1.172340,1.172340,38.394956
1,2021-04-01 07:15:00,1.17315,1.17321,1.17274,1.17297,1.17329,1.17334,1.17287,1.17310,1.17322,1.17328,1.17281,1.17304,490,1.172337,1.172357,1.172398,1.172398,46.548073
2,2021-04-01 07:20:00,1.17298,1.17362,1.17263,1.17362,1.17311,1.17375,1.17276,1.17375,1.17304,1.17368,1.17270,1.17368,548,1.172422,1.172448,1.172500,1.172500,49.272128
3,2021-04-01 07:25:00,1.17364,1.17420,1.17352,1.17414,1.17377,1.17433,1.17366,1.17427,1.17370,1.17426,1.17359,1.17420,698,1.172530,1.172563,1.172630,1.172630,50.893711
4,2021-04-01 07:30:00,1.17412,1.17425,1.17381,1.17394,1.17425,1.17438,1.17391,1.17405,1.17418,1.17432,1.17386,1.17400,398,1.172614,1.172653,1.172731,1.172731,53.478762


In [7]:
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema200,ema100,ema50,ema25,adx
74764,2022-04-01 05:35:00,1.10690,1.10694,1.10668,1.10676,1.10707,1.10710,1.10683,1.10691,1.10698,1.10702,1.10676,1.10684,291,1.107953,1.106952,1.106711,1.106711,9.813219
74765,2022-04-01 05:40:00,1.10675,1.10727,1.10672,1.10720,1.10691,1.10744,1.10689,1.10737,1.10683,1.10735,1.10680,1.10728,400,1.107946,1.106958,1.106733,1.106733,9.807821
74766,2022-04-01 05:45:00,1.10721,1.10722,1.10698,1.10704,1.10737,1.10738,1.10715,1.10721,1.10729,1.10730,1.10707,1.10712,183,1.107938,1.106961,1.106749,1.106749,10.947864
74767,2022-04-01 05:50:00,1.10703,1.10729,1.10700,1.10726,1.10721,1.10744,1.10717,1.10743,1.10712,1.10736,1.10708,1.10734,227,1.107932,1.106969,1.106772,1.106772,12.020967
74768,2022-04-01 05:55:00,1.10727,1.10737,1.10667,1.10673,1.10742,1.10754,1.10683,1.10690,1.10734,1.10745,1.10675,1.10682,439,1.107921,1.106966,1.106774,1.106774,13.107870


In [8]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [9]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100

In [16]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, each_bar, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, invert, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    win_amounts, loss_amounts, pips_risked = [], [], []
    day_fees = 0
    trade = None
    pip_movement /= divider
    pips_to_risk = pips_to_risk / divider if type(pips_to_risk) != str else pips_to_risk

    if n_bars == 'any':
       lookback, lookforward = 12, 0

    else:
        lookback = n_bars + 1 if use_pullback else n_bars
        lookforward = -1 if use_pullback else 0

    for i in range(lookback, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_ask_close, curr_bid_close, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Ask_Close', 'Bid_Close', 'Date']]
        spread = abs(curr_ask_open - curr_bid_open)

        if n_bars == 'any':
            mid_opens = list(df.loc[df.index[i - lookback:i], 'Mid_Open'])
            mid_highs = list(df.loc[df.index[i - lookback:i], 'Mid_High'])
            mid_lows = list(df.loc[df.index[i - lookback:i], 'Mid_Low'])
            mid_closes = list(df.loc[df.index[i - lookback:i], 'Mid_Close'])
            buy_signal, sell_signal = False, False
            buy_pip_movement, sell_pip_movement = 0, 0
            
            for j in range(len(mid_opens) - 1, -1, -1):
                if mid_opens[j] < mid_closes[j]:
                    buy_pip_movement += abs(mid_opens[j] - mid_closes[j])

                    if buy_pip_movement >= pip_movement:
                        buy_signal = True
                        break

                else:
                    break

            for j in range(len(mid_opens) - 1, -1, -1):
                if mid_opens[j] > mid_closes[j]:
                    sell_pip_movement += abs(mid_opens[j] - mid_closes[j])

                    if sell_pip_movement >= pip_movement:
                        sell_signal = True
                        break

                else:
                    break

        else:
            mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
            mid_highs = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_High'])
            mid_lows = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Low'])
            mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

            if each_bar:
                buy_signal = all([mid_opens[j] < mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement for j in range(len(mid_opens))])
                sell_signal = all([mid_opens[j] > mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement for j in range(len(mid_opens))])

            else:
                buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
                sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

        if use_pullback and buy_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        if use_pullback and sell_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

        if ema is not None:
            curr_ema, curr_mid_close = df.loc[df.index[i - 1], [ema, 'Mid_Close']]
            buy_signal = buy_signal and curr_mid_close > curr_ema
            sell_signal = sell_signal and curr_mid_close < curr_ema

        curr_adx = df.loc[df.index[i - 1], 'adx']
        moving_enough = curr_adx > adx_cutoff

        buy_signal = buy_signal and moving_enough
        sell_signal = sell_signal and moving_enough

        if invert:
            buy_signal, sell_signal = sell_signal, buy_signal

        highest_high, lowest_low = max(mid_highs), min(mid_lows)

        within_time_slot = start_hour <= curr_date.hour <= end_hour

        if trade is None and within_time_slot:
            if buy_signal:
                open_price = float(curr_ask_open)

                stop_loss = open_price - pips_to_risk if type(pips_to_risk) != str else lowest_low
                stop_loss = round(stop_loss, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                 'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                 'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 
                                 'end_date': None, 'prev_profit_ratio': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(curr_bid_open)
                
                stop_loss = open_price + pips_to_risk if type(pips_to_risk) != str else highest_high
                stop_loss = round(stop_loss, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 
                                'end_date': None, 'prev_profit_ratio': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)


        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and use_trailing_sl and curr_bid_close > trade['open_price']:
            curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'buy' and not use_trailing_sl and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and use_trailing_sl and curr_ask_close < trade['open_price']:
            curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'sell' and not use_trailing_sl and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts,

In [20]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5]
spread_cutoffs = [0.10]
each_bar_vals = [False]
n_bars_vals = ['any']  # LEAVE THE 'any' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pip_movement_vals = [20]
use_pullback_vals = [False]
pips_to_risk_vals = ['bars', 10, 20]  # LEAVE THE 'bars' VALUE IN, YOU CAN CHANGE THE OTHER VALUES
pullback_percentages = [0.35, 0.50]
invert_vals = [True, False]
ema_vals = [None, 'ema200', 'ema100', 'ema50', 'ema25']  # 'None' means it won't use the EMA
adx_cutoffs = [0, 25, 30]  # ADX is between 0 and 100; usually ADX > 25 means the market is moving fairly well

# Hour values are between 0 (12 AM/midnight) and 23 (11 PM).  The dates are in UTC, which is about 7 hours ahead of CA time.
# For example, if you wanted to only place trades between 6 AM and 12 PM CA time, the start hour should be 13 (1 PM UTC) and
# the end hour should be 19 (7 PM UTC).  If you want to trade all 24 hours of the day, you would set the start hour to 0 and
# the end hour to 23.
start_hours = [13, 14]
end_hours = [19, 20]
use_trailing_sl_vals = [True, False]

all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for each_bar in each_bar_vals:
            for n_bars in n_bars_vals:
                for pip_movement in pip_movement_vals:
                    for use_pullback in use_pullback_vals:
                        for pips_to_risk in pips_to_risk_vals:
                            for pullback_percentage in pullback_percentages:
                                for invert in invert_vals:
                                    for ema in ema_vals:
                                        for adx_cutoff in adx_cutoffs:
                                            for start_hour in start_hours:
                                                for end_hour in end_hours:
                                                    for use_trailing_sl in use_trailing_sl_vals:
                                                        all_combos.append((risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, invert, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_each_bar_val = None
best_n_bars_val = None
best_pip_movement_val = None
best_use_pullback_val = None
best_pips_to_risk = None
best_pullback_percentage = None
best_invert_val = None
best_ema = None
best_adx_cutoff = None
best_start_hour = None
best_end_hour = None
best_use_trailing_sl_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, invert, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts, = run_simulation(risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback, pips_to_risk, pullback_percentage, invert, ema, adx_cutoff, start_hour, end_hour, use_trailing_sl)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'each_bar': each_bar, 'n_bars': n_bars, 'pip_movement': pip_movement, 'use_pullback': use_pullback, 'pips_to_risk': pips_to_risk, 'pullback_percentage': pullback_percentage, 'invert': invert, 'ema': ema, 'adx_cutoff': adx_cutoff, 'start_hour': start_hour, 'end_hour': end_hour, 'use_trailing_sl': use_trailing_sl})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_each_bar_val = each_bar
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_use_pullback_val = use_pullback
        best_pips_to_risk = pips_to_risk
        best_pullback_percentage = pullback_percentage
        best_invert_val = invert
        best_ema = ema
        best_adx_cutoff = adx_cutoff
        best_start_hour = start_hour
        best_end_hour = end_hour
        best_use_trailing_sl_val = use_trailing_sl

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 12

-2136.9145200000016
Num buys: 135
Num sells: 136
Num trades: 271
Num wins: 57
Num losses: 163
Win streak: 6
Loss streak: 14
Avg pips risked: 0.0020000000000000018
Avg win amount: 109.64473684210536
Min win amount: 49.99800000000005
Max win amount: 349.98600000000033
Avg loss amount: -38.08258878504676
Min loss amount: -49.99800000000005
Max loss amount: 0.0
Remaining runs: 11
Best reward so far: -2136.9145200000016

174.8692499999962
Num buys: 10
Num sells: 16
Num trades: 26
Num wins: 8
Num losses: 11
Win streak: 3
Loss streak: 7
Avg pips risked: 0.0027596153846153925
Avg win amount: 93.74778500000005
Min win amount: 49.99857999999967
Max win amount: 249.9934999999986
Avg loss amount: -30.554769444444688
Min loss amount: -49.999620000007006
Max loss amount: 0.0
Remaining runs: 10
Best reward so far: 174.8692499999962

-2128.9148400000017
Num buys: 135
Num sells: 136
Num trades: 271
Num wins: 57
Num losses: 163
Win streak: 6
Loss streak: 14
Avg pips risked: 0.0020000000000

KeyboardInterrupt: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best pullback percentage: ' + str(best_pullback_percentage))
print('Best invert val: ' + str(best_invert_val))
print('Best ema: ' + str(best_ema))
print('Best adx cutoff: ' + str(best_adx_cutoff))
print('Best start hour: ' + str(best_start_hour))
print('Best end hour: ' + str(best_end_hour))
print('Best use trailing sl val: ' + str(best_use_trailing_sl_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

: 